# 7.0.0 Building a Final Classifier Model

Methodology

The culmination of the modeling process involves the construction and deployment of the final classifier model using XGBoost. The final model incorporates a set of finely tuned parameters to optimize performance, particularly focusing on handling the imbalanced nature of the dataset. The parameters were selected based on previous optimization efforts, ensuring that the model is robust, generalizes well on unseen data, and maximizes the AUC metric.

Model Parameters:



In [1]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
from xgboost import XGBClassifier

from src.utils import calculate_metrics, save_pickle

### 1. Loan Data


In [2]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    
model_parameters = config["model_parameters"]["xgbm"]
numeric_features = config["filter_features"]["numerical"]
features = numeric_features
target = config["main"]["target"]
data_train_path = Path.cwd().parent / config["main"]["data_train_path"]
train_validation_path = Path.cwd().parent / config["main"]["data_validation_path"]

train_df = pd.read_pickle(data_train_path)
validation_df = pd.read_pickle(train_validation_path)

X_train, Y_train = train_df[features], train_df[target]
X_valid, Y_valid = validation_df[features], validation_df[target]

split_seed = config["main"]["random_seed"]

X_train.shape

(9479, 14)

### 2. Train Model

In [3]:
xgbm_model = XGBClassifier(missing=np.nan, **model_parameters, random_state=split_seed)

xgbm_model.fit(X_train, Y_train)
xgbm_preds = xgbm_model.predict_proba(X_valid)[:, 1]

model_results = calculate_metrics(Y_valid, xgbm_preds)
model_results

/Users/mfrancys/.local/share/virtualenvs/credit-risk-machine-learning-pipeline-TF6eVTA0/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [01:27:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "booster"", "eval_metric"", "objective"" } are not used.

  warnings.warn(smsg, UserWarning)


{'roc_auc_score': 0.6110941648308197,
 'pr_auc': 0.28320533857646,
 'ks': 0.17798678190137265}

### 3. Save Model Classifier Object 

In [4]:
dataset_date = str(train_validation_path).split("/")[-1:][0][0:6]
model_train_path = Path.cwd().parent / f"models/develop/{dataset_date}_xgbm_classifier.pickle"

save_pickle(xgbm_model, model_train_path)